# Testing covid and data 

In [ ]:
import requests
import pandas as pd
from datetime import datetime

# entity_name = 'INSTITUTO NACIONAL DE VÍAS (INVIAS)'
# url_secop_i = 'https://www.datos.gov.co/resource/c82b-7jfi.json'
# p_entity = {'nombre_de_la_entidad': entity_name,
#             '$limit': '10000',
#             'causal_de_otras_formas_de': 'Contratos Interadministrativos (Literal C)'}
# r_entity = requests.get(url_secop, params=p_entity)
# d_entity = r_entity.json()  # To .json
# df_entity = pd.DataFrame(d_entity)  # To df
# df_entity

# Extraction
SECOP I

In [ ]:
# SECOP I
url_secop_i = 'https://www.datos.gov.co/resource/c82b-7jfi.json'
p_secop_i = {'$limit': '1000000',
            'anno_firma_del_contrato': '2020'}
r_secop_i = requests.get(url_secop_i, params=p_secop_i)
d_secop_i = r_secop_i.json()  # To .json
secop_1 = pd.DataFrame(d_secop_i)  # To df
print(len(secop_1))
secop_1.head(2)

In [ ]:
secop_1.columns

In [ ]:
covid_pattern = 'covid|coronavirus|pandemia|emergencia'

In [ ]:
# secop_1['detalle_lower'] = secop_1['detalle_del_objeto_a_contratar'].str.lower()

#secop_1.loc[lambda x: x.detalle_lower.str.contains(covid_pattern)].detalle_lower.to_csv('../data/secop_1_descriptions.csv', index=False, header=False)

SECOP II

In [ ]:
# SECOP II
url_secop_ii = "https://www.datos.gov.co/resource/jbjy-vk9h.json?$where=fecha_de_firma>='2020-01-01'"
p_secop_ii = {'$limit': '1000000'}
r_secop_ii = requests.get(url_secop_ii, params=p_secop_ii)
d_secop_ii = r_secop_ii.json()  # To .json
secop_2 = pd.DataFrame(d_secop_ii)  # To df
print(len(secop_2))
secop_2.head(2)

In [ ]:
secop_2 = (secop_2
    .assign(descripcion_del_proceso = lambda x: x.descripcion_del_proceso.str.lower(),
            fecha_de_firma = lambda x: pd.to_datetime(x.fecha_de_firma))
)

In [ ]:
secop_2.loc[lambda x: x.descripcion_del_proceso.str.lower().str.contains(covid_pattern)].shape[0]

In [ ]:
secop_2.shape

# Unification

In [ ]:
column_map = {
    'nombre_de_la_entidad': 'nombre_entidad',
    'nit_de_la_entidad': 'nit_entidad',
    'departamento_entidad': 'departamento',
    'municipio_entidad': 'ciudad',
    'nivel_entidad': 'orden',
    'numero_de_proceso': 'proceso_de_compra',
    'numero_del_contrato': 'id_contrato',
    'estado_del_proceso': 'estado_contrato',
    'detalle_del_objeto_a_contratar': 'descripcion_del_proceso',
    'tipo_de_contrato': 'tipo_de_contrato',
    'tipo_de_proceso': 'modalidad_de_contratacion',
    'fecha_de_firma_del_contrato': 'fecha_de_firma',
    'fecha_ini_ejec_contrato': 'fecha_de_inicio_de_ejecucion',
    'fecha_fin_ejec_contrato': 'fecha_de_fin_de_ejecucion',
    'tipo_identifi_del_contratista': 'tipodocproveedor',
    'identificacion_del_contratista': 'documento_proveedor',
    'nom_raz_social_contratista': 'proveedor_adjudicado',
    'cuantia_contrato': 'valor_del_contrato',
    'espostconflicto': 'espostconflicto',
    'ruta_proceso_en_secop_i': 'urlproceso',
    'origen_de_los_recursos': 'origen_de_los_recursos',
}

In [ ]:
#rename columns from covid 1 to match covid 2
secop_1 = secop_1.rename(columns=column_map)

In [ ]:
# source
secop_1['source'] = 'secop_1'
secop_2['source'] = 'secop_2'

In [ ]:
# unification
union_cols = list(column_map.values())+['source']

secop_union = pd.concat([secop_2, secop_1])
secop_union = secop_union[union_cols]

## preprocessing

In [ ]:
secop_union.dtypes

In [ ]:
secop_union['descripcion_del_proceso'] = secop_union.descripcion_del_proceso.str.lower()

In [ ]:
secop_union['fecha_de_firma'] = pd.to_datetime(secop_union.fecha_de_firma)
secop_union['fecha_de_inicio_de_ejecucion'] = pd.to_datetime(secop_union.fecha_de_inicio_de_ejecucion)
secop_union['fecha_de_fin_de_ejecucion'] = pd.to_datetime(secop_union.fecha_de_fin_de_ejecucion)

In [ ]:
secop_union['url'] = secop_union.urlproceso.apply(lambda x: x['url'])

In [ ]:
secop_union['is_covid'] = secop_union.descripcion_del_proceso.str.contains(covid_pattern)

In [ ]:
# saving data 
secop_union.to_pickle('../data/secop_union_all.pickle')
secop_union.to_csv('../data/secop_union_all.csv')

In [ ]:
secop_1.to_pickle('../data/secop_1.pickle')
secop_1.to_csv('../data/secop_1.csv')
secop_2.to_pickle('../data/secop_2.pickle')
secop_2.to_csv('../data/secop_2.csv')

In [ ]:
secop_union.dtypes

In [ ]:
# covid_separation
secop_1_covid = secop_1.loc[lambda x: x.detalle_del_objeto_a_contratar.str.contains(covid_pattern)]

In [ ]:
# covid flag
secop_1.assign(is_covid = lambda x: x.detalle_del_objeto_a_contratar.str.)

In [ ]:
secop_1_covid.shape[0]

In [ ]:
secop_1_covid.reset_index(drop=True).to_feather('../data/secop_1_covid.feather')
secop_1_covid.reset_index(drop=True).to_pickle('../data/secop_1_covid.pkl')

In [ ]:
secop_2_covid = secop_2.loc[lambda x: x.descripcion_del_proceso.str.contains('covid|coronavirus|pandemia|emergencia')]

In [ ]:
secop_2_covid.shape[0]

In [ ]:
secop_2_covid.reset_index(drop=True).to_feather('../data/secop_2_covid.feather')
secop_2_covid.reset_index(drop=True).to_pickle('../data/secop_2_covid.pkl')

In [ ]:
secop_un